# A/B Testing the Udacity Website

In these exercises, we'll be analyzing data on user behavior from an experiment run by Udacity, the online education company. More specifically, we'll be looking at a test Udacity ran to improve the onboarding process on their site.

Udacity's test is an example of an "A/B" test, in which some portion of users visiting a website (or using an app) are randomly selected to see a new version of the site. An analyst can then compare the behavior of users who see a new website design to users seeing their normal website to estimate the effect of rolling out the proposed changes to all users. While this kind of experiment has it's own name in industry (A/B testing), to be clear it's just a randomized experiment, and so everything we've learned about potential outcomes and randomized experiments apply here. 

(Udacity has generously provides the data from this test under an Apache open-source license, and you can find their [original writeup here](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook). If you're interested in learning more on A/B testing in particular, it seems only fair while we use their data to flag they have a full course on the subject [here](https://www.udacity.com/course/ab-testing--ud257).)

## Udacity's Test

The test [is described by Udacity as follows](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook): 

At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials".

**Current Conditions Before Change**

- If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first.
- If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.

**Description of Experimented Change**

- In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course.
- If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.
- At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. This [screenshot](images/udacity_checkyoureready.png) shows what the experiment looks like.

**Udacity's Hope is that...**:

> this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time -- without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_abtesting.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}
```


### Submission Limits

Please remember that you are **only allowed FOUR submissions to the autograder.** Your last submission (if you submit 4 or fewer times), or your third submission (if you submit more than 4 times) will determine your grade Submissions that error out will **not** count against this total.

That's one more than usual in case there are issues with exercise clarity.

In [1]:
import warnings
import pandas as pd
from scipy.stats import ttest_ind
import statsmodels.formula.api as smf


warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option("mode.copy_on_write", True)

results = dict()

## Import the Data

### Exercise 1

Begin by importing Udacity's data on user behavior [here.](https://github.com/nickeubank/MIDS_Data/tree/master/udacity_AB_testing) 

There are TWO datasets for this test — one for the control data (users who saw the original design), and one for treatment data (users who saw the experimental design). Udacity decided to show their test site to 1/2 of visitors, so there are roughly the same number of users appearing in each dataset (though this is not a requirement of AB tests).

Please remember to load the data directly from github to assist the autograder.

In [2]:
path_control_data = "https://media.githubusercontent.com/media/nickeubank/MIDS_Data/master/udacity_AB_testing/control_data.csv"
path_experiment_data = "https://media.githubusercontent.com/media/nickeubank/MIDS_Data/master/udacity_AB_testing/experiment_data.csv"

control_data = pd.read_csv(path_control_data)
experiment_data = pd.read_csv(path_experiment_data)

### Exercise 2

Explore the data. Can you identify the unit of observation of the data (e.g. what is represented by each row)?

(Note this is not the only way that A/B test data can be collected and/or reported — this is just what Udacity provided, presumably to help address privacy concerns.)

In [3]:
print(control_data.head())
print()
print(control_data.info())
print()
print(control_data.describe())

          Date  Pageviews  Clicks  Enrollments  Payments
0  Sat, Oct 11       7723     687        134.0      70.0
1  Sun, Oct 12       9102     779        147.0      70.0
2  Mon, Oct 13      10511     909        167.0      95.0
3  Tue, Oct 14       9871     836        156.0     105.0
4  Wed, Oct 15      10014     837        163.0      64.0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         37 non-null     object 
 1   Pageviews    37 non-null     int64  
 2   Clicks       37 non-null     int64  
 3   Enrollments  23 non-null     float64
 4   Payments     23 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB
None

          Pageviews      Clicks  Enrollments    Payments
count     37.000000   37.000000    23.000000   23.000000
mean    9339.000000  766.972973   164.565217   88.391304
std      740.239563   

In [4]:
print(experiment_data.head())
print()
print(experiment_data.info())
print()
print(experiment_data.describe())

          Date  Pageviews  Clicks  Enrollments  Payments
0  Sat, Oct 11       7716     686        105.0      34.0
1  Sun, Oct 12       9288     785        116.0      91.0
2  Mon, Oct 13      10480     884        145.0      79.0
3  Tue, Oct 14       9867     827        138.0      92.0
4  Wed, Oct 15       9793     832        140.0      94.0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         37 non-null     object 
 1   Pageviews    37 non-null     int64  
 2   Clicks       37 non-null     int64  
 3   Enrollments  23 non-null     float64
 4   Payments     23 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB
None

          Pageviews      Clicks  Enrollments    Payments
count     37.000000   37.000000    23.000000   23.000000
mean    9315.135135  765.540541   148.826087   84.565217
std      708.070781   

> We can see that both datasets consist of 37 rows, with daily information from October 11th to November 16th.
>
>However, for the Enrollment and Payments variables, data is only available from October 11th to November 2nd in both datasets, with null values for subsequent periods in these two variables.
>
>`The unit of observation is days`, in other words, each row represents aggregated information for the Pageviews, Clicks, Enrollments, and Payments variables per day.

## Pick your measures

### Exercise 3

The easiest way to analyze this data is to stack it into a single dataset where each observation is a day-treatment-arm (so you should end up with two rows per day, one for those who are in the treated groups, and one for those who were in the control group). Note that currently nothing in the data identifies whether a given observation is a treatment group observation or a control group observation, so you'll want to make sure to add a "treatment" indicator variable.

The variables in the data are:

- Pageviews: number of unique users visiting homepage
- Clicks: number of those users clicking "Start Free Trial"
- Enrollments: Number of people enrolling in trial
- Payments: Number of people who eventually pay for the service. Note the `payment` column reports payments for the users who first visited the site on the reported date, not payments occurring on the reported date.

In [5]:
control_data["treatment"] = "control"
experiment_data["treatment"] = "experiment"

data = pd.concat([control_data, experiment_data], ignore_index=True)

### Exercise 4

Given Udacity's goals, what outcome are they hoping will be impacted by their manipulation?

Or, to ask the same question in the language of the Potential Outcomes Framework, what is their $Y$?

Or to ask the same question in the language of Kohavi, Tang and Xu, what is their *Overall Evaluation Criterion (OEC)*?

(I'm only asking one question, I'm just trying to phrase it using different terminologies we've encountered to help you see how they all fit together)

When you feel like you have your answer, please compute it. Store the average value of the variable in `results` under the key `ex4_avg_oec`. **Please round your answer to 4 decimal places.**

NOTE: You'll probably notice you have two choices to make when it comes to actually computing the OEC. 

- You could probably imagine either computing a ratio or a difference of two things — please calculate the difference.
- You may also be unsure whether to normalize by `Clicks`. Normalizing by clicks will help account for variation that comes from day-to-day variation in users, so it's a good thing to do. With infinite data, you'd expect to get the same results without normalizing by `Clicks` (since on average the same share of users are in each arm of the experiment), but for finite data it's a good strategy. Note that this is only ok because users make the choice to click or not *before* they see different versions of the website (it is "pre-treatment").

Just to make sure you're on track, your measure should have an average value of *about* 9%.

>Given Udacity's goals, the outcome they are hoping will be impacted by their manipulation, or their OEC, is the daily churn ratio. This ratio represents the number of people who enroll in the free trial but ultimately do not pay for the service, normalized by clicks.
>
>Udacity's objective is to reduce this churn ratio with the modification in the enrollment process.

In [6]:
data["oec_churn_ratio"] = (data["Enrollments"] - data["Payments"]) / data["Clicks"]
ex4_avg_oec = data["oec_churn_ratio"].mean()
results["ex4_avg_oec"] = round(ex4_avg_oec, 4)
print(
    f"In our dataset, the average Overall Evaluation Criterion (OEC) or average churn ratio, normalized by clicks, is {ex4_avg_oec:.4f}"
)

In our dataset, the average Overall Evaluation Criterion (OEC) or average churn ratio, normalized by clicks, is 0.0941


### Exercise 5

Given Udacity's goals, what outcome are they hoping will *not* be impacted by their manipulation? In other words, what do they want to measure to ensure their treatment doesn't have unintended negative consequences that might be really costly to their operation?

Note that while this isn't how Kohavi, Tang, and Xu use the term "guardrail metrics" — they usually use the term to refer to things we measure to ensure the experiment is working the way it should — some people would also use the term "guardrail metrics" for something that could be impacted even if the experiment is working correctly, but which the organization wants to track to ensure they aren't impacted because they are deemed really important.

Again, please normalize by `Clicks`. Store the average value of this guardrail metric as `ex5_avg_guardrail` and **round your answer to 4 decimal places.**

> Given Udacity's goals, they hope that their manipulation will not significantly affect the number of students who continue past the free trial and eventually complete the course. 
>
>In the case of our available information, we will use the daily number of payments (payments made by users who first visited the site on the reported date, not payments occurring on the reported date), normalized by the number of clicks

In [7]:
data["guardrail_payments_ratio"] = data["Payments"] / data["Clicks"]
ex5_avg_guardrail = data["guardrail_payments_ratio"].mean()
results["ex5_avg_guardrail"] = round(ex5_avg_guardrail, 4)
print(
    f"In our dataset, the average guardrail metric, which represents the ratio of payments to clicks, is {ex5_avg_guardrail:.4f}"
)

In our dataset, the average guardrail metric, which represents the ratio of payments to clicks, is 0.1158


## Validating The Data

### Exercise 6

Whenever you are working with experimental data, the first thing you want to do is verify that users actually were randomly sorted into the two arms of the experiment. In this data, half of users were supposed to be shown the old version of the site and half were supposed to see the new version.

`Pageviews` tells you how many unique users visited the welcome site we are experimenting on. `Pageviews` is what is sometimes called an "invariant" or "guardrail" variable, meaning that it shouldn't vary across treatment arms—after all, people have to visit the site before they get a chance to see the treatment, so there's no way that being assigned to treatment or control should affect the number of pageviews assigned to each group.

"Invariant" variables are also an example of what are known as a "pre-treatment" variable, because pageviews are determined before users are manipulated in any way. That makes it analogous to gender or age in experiments where you have demographic data—a person's age and gender are determined before they experience any manipulations, so the value of any pre-treatment attributes should be the same across the two arms of our experiment. This is what we've previously called "checking for balance," If pre-treatment attributes aren't balanced, then we may worry our attempt to randomly assign people to different groups failed.  Kohavi, Tang and Xu call this a "trust-based guardrail metric" because it helps us determine if we should trust our data.

To test the quality of the randomization, calculate the average number of pageviews for the treated group and for the control group. Do they look similar?


In [8]:
control_pageviews_avg = data[data["treatment"]
                             == "control"]["Pageviews"].mean()
experiment_pageviews_avg = data[data["treatment"]
                                == "experiment"]["Pageviews"].mean()
print(
    f"Average number of pageviews for the control group: {control_pageviews_avg:.1f}")
print(
    f"Average number of pageviews for the experiment group: {experiment_pageviews_avg:.1f}"
)

Average number of pageviews for the control group: 9339.0
Average number of pageviews for the experiment group: 9315.1


> The average number of pageviews for the control group is 9,339, while for the experiment group it is 9,315. These numbers fall within the same range, suggesting similarity. However, in the following question, we will assess whether the differences between both groups are significant or not.

### Exercise 7

"Similar" is a tricky concept -- obviously, we expect *some* differences across groups since users were *randomly* divided across treatment arms. The question is whether the differences between groups are larger than we'd expect to emerge given our random assignment process. To evaluate this, let's use a `ttest` to test the statistical significance of the differences we see. 

**Note**: Remember that scipy functions don't accept `pandas` objects, so you use a scipy function, you have to pass the numpy vectors underlying your data with the `.values` operator (e.g. `df.my_column.values`). 

Does the difference in `pageviews` look statistically significant?

Store the resulting p-value in `ex7_ttest_pvalue` **rounded to four decimal places.**

In [9]:
control_pageviews = data[data["treatment"] == "control"]["Pageviews"]
experiment_pageviews = data[data["treatment"] == "experiment"]["Pageviews"]

t_statistic, p_value = ttest_ind(control_pageviews.values, experiment_pageviews.values)

ex7_ttest_pvalue = p_value
results["ex7_ttest_pvalue"] = round(ex7_ttest_pvalue, 4)

print(
    f"The p-value from the t-test for the difference in pageviews between the control and experiment groups is: {ex7_ttest_pvalue:.4f}"
)

The p-value from the t-test for the difference in pageviews between the control and experiment groups is: 0.8877


> Based on the p-value from the t-test for the difference in pageviews between the control and experiment groups (p = 0.89), we do not have sufficient evidence to reject the null hypothesis. Therefore, the difference in pageviews does not appear to be statistically significant. 

### Exercise 8

`Pageviews` is not the only "pre-treatment" variable in this data we can use to evaluate balance/use as a guardrail metric. What other measure is pre-treatment? Review the description of the experiment if you're not sure.

> Another 'pre-treatment' variable that we can use to evaluate balance/use as a guardrail metric could be the number of clicks, as these represent the number of users clicking 'Start Free Trial', which is a step prior to the experiment. Therefore, we would expect the value of clicks in the treatment and control groups to be similar.

### Exercise 9

Check if the other pre-treatment variable is also balanced. Store the p-value of your test of difference in `results` under the key `"ex9_ttest_pvalue_clicks"` **rounded to four decimal places.**

In [10]:
control_clicks = data[data["treatment"] == "control"]["Clicks"]
experiment_clicks = data[data["treatment"] == "experiment"]["Clicks"]

control_clicks_avg = control_clicks.mean()
experiment_clicks_avg = experiment_clicks.mean()

t_statistic, p_value = ttest_ind(
    control_clicks.values, experiment_clicks.values)

ex9_ttest_pvalue_clicks = p_value
results["ex9_ttest_pvalue_clicks"] = round(ex9_ttest_pvalue_clicks, 4)

print(
    f"Average number of clicks for the control group: {control_clicks_avg:.1f}")
print(
    f"Average number of clicks for the experiment group: {experiment_clicks_avg:.1f}")
print(
    f"\nThe p-value from the t-test for the difference in clicks between the control and experiment groups is: {ex9_ttest_pvalue_clicks:.4f}"
)

Average number of clicks for the control group: 767.0
Average number of clicks for the experiment group: 765.5

The p-value from the t-test for the difference in clicks between the control and experiment groups is: 0.9264


> This high p-value (0.93) suggests that there is no significant difference in the number of clicks between the two groups. Combined with the observation of similar pageviews between the control and experiment groups, it appears that the randomization process was effective in creating balanced groups. Therefore, we have confidence in the validity of the randomization process, ensuring that the treatment and control groups are comparable in terms of their characteristics at baseline.

## Estimating the Effect of Experiment

### Exercise 10

Now that we've validated our randomization, our next task is to estimate our treatment effect. First, though, there's an issue with your data you've been able to largely ignore until now, but which you should get a grip on before estimating your treatment effect — can you tell what it is and what you should do about it?

Store the number of observations in your data *after* you've addressed this in `ex10_num_obs` (this is mostly meant as a way to sanity check your answer with autograder).

> As mentioned previously, the data for the 'Enrollment' and 'Payments' variables is only available from October 11th to November 2nd, with null values for subsequent periods in these two variables. 
>
> Given that these variables are used to calculate our Overall Evaluation Criterion, we will remove records with null data for periods after November 2nd.

In [11]:
data.dropna(inplace=True)
ex10_num_obs = data.shape[0]
results["ex10_num_obs"] = ex10_num_obs

print(
    f"The number of observations in the data after addressing the missing data issue is: {ex10_num_obs}"
)

The number of observations in the data after addressing the missing data issue is: 46



### Exercise 11

Now that we've established we have good balance (meaning we think randomization was likely successful), we can evaluate the effects of the experiment. Test whether the OEC and the metric you *don't* want affected have different average values in the control group and treatment group. 

Because we've randomized, this is a consistent estimate of the Average Treatment Effect of Udacity's website change.

Calculate the difference in means in your OEC and guardrail metrics using a simple t-test. Store the resulting effect estimates in `ex11_oec_ate` and `ex11_guard_ate` and p-values in `ex11_oec_pvalue` and `ex11_guard_pvalue`. **Please round all answers to 4 decimal places.** Report your ATE in *percentage points*, where `1` denotes 1 percentage point.


In [12]:
data.head()

,Date,Pageviews,Clicks,Enrollments,Payments,treatment,oec_churn_ratio,guardrail_payments_ratio
0,"Sat, Oct 11",7723,687,134.0,70.0,control,0.093159,0.101892
1,"Sun, Oct 12",9102,779,147.0,70.0,control,0.098845,0.089859
2,"Mon, Oct 13",10511,909,167.0,95.0,control,0.079208,0.104510
3,"Tue, Oct 14",9871,836,156.0,105.0,control,0.061005,0.125598
4,"Wed, Oct 15",10014,837,163.0,64.0,control,0.118280,0.076464


In [13]:
# OEC

control_group_oec = data[data["treatment"] == "control"]["oec_churn_ratio"]
experiment_group_oec = data[data["treatment"]
                            == "experiment"]["oec_churn_ratio"]

t_statistic_oec, pvalue_oec = ttest_ind(
    experiment_group_oec, control_group_oec)

ex11_oec_ate = -(experiment_group_oec.mean() - control_group_oec.mean()) * 100
ex11_oec_pvalue = pvalue_oec

results["ex11_oec_ate"] = round(ex11_oec_ate, 4)
results["ex11_oec_pvalue"] = round(ex11_oec_pvalue, 4)

In [14]:
# Guardrail metric

control_group_guard = data[data["treatment"] == "control"]["guardrail_payments_ratio"]
experiment_group_guard = data[data["treatment"] == "experiment"][
    "guardrail_payments_ratio"
]

t_statistic_guard, pvalue_guard = ttest_ind(experiment_group_guard, control_group_guard)

ex11_guard_ate = -(experiment_group_guard.mean() - control_group_guard.mean()) * 100
ex11_guard_pvalue = pvalue_guard

results["ex11_guard_ate"] = round(ex11_guard_ate, 4)
results["ex11_guard_pvalue"] = round(ex11_guard_pvalue, 4)

In [15]:
print("OEC:")
print(f"Average Treatment Effect: {ex11_oec_ate:.4f} percentage points")
print(f"p-value: {ex11_oec_pvalue:.4f}")

print("\nGuardrail metric:")
print(f"Average Treatment Effect: {ex11_guard_ate:.4f} percentage points")
print(f"p-value: {ex11_guard_pvalue:.4f}")

OEC:
Average Treatment Effect: 1.5888 percentage points
p-value: 0.1319

Guardrail metric:
Average Treatment Effect: 0.4897 percentage points
p-value: 0.5928


### Exercise 12

Do you feel that Udacity achieved their goal? Did their intervention cause them any problems? If they asked you "What would happen if we rolled this out to everyone?" what would you say?

As you answer this question, a small additional question: up until this point you've (presumably) been reporting the default p-values from the tools you are using. These, as you may recall from stats 101, are two-tailed p-values. Do those seem appropriate for your OEC?

>It doesn't seem like Udacity achieved its objective. While the intervention didn't significantly reduce the number of students continuing past the free trial (as indicated by the guardrail metric), it also didn't effectively reduce the number of frustrated students leaving the free trial due to time constraints or our (churn ratio/ OEC).
>
>The average treatment effect on the OEC was -1.59 percentage points, but the associated p-value was 0.13, indicating that this difference wasn't statistically significant. On the other hand, the average treatment effect on the guardrail metric was -0.49 percentage points, with a p-value of 0.59, suggesting that there wasn't a significant difference in this metric either.
>
>Given the relatively high p-values for both metrics, we can't confidently conclude that the intervention had a significant impact on either of the evaluated metrics.
>
>Although in this exercise we have been using the default p-values, these values are two-tailed p-values. Since we are actually interested in whether our churn ratio (OEC) is reduced for the treatment group and if our payment ratio (guardrail metric) is not reduced, it would be more appropriate to use a one-tailed approach in our hypothesis testing. A one-tailed approach would allow us to specifically evaluate whether there is a reduction in the churn ratio or if there is no reduction in the payment ratio, which aligns better with our research question and study objective.

### Exercise 13

One of the magic things about experiments is that all you have to do is compare averages to get an average treatment effect. However, you *can* do other things to try and increase the statistical power of your experiments, like add controls in a linear regression model. 

As you likely know, a bivariate regression is exactly equivalent to a t-test, so let's start by re-estimating the effect of treatment on your OEC using a linear regression. Can you replicate the results from your t-test? They shouldn't just be close—they should be numerically equivalent (i.e. exactly the same to the limits of floating point number precision). 

In [16]:
model = smf.ols("oec_churn_ratio ~ treatment", data=data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        oec_churn_ratio   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.356
Date:                Mon, 18 Mar 2024   Prob (F-statistic):              0.132
Time:                        12:10:25   Log-Likelihood:                 89.832
No. Observations:                  46   AIC:                            -175.7
Df Residuals:                      44   BIC:                            -172.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [17]:
print(
    f"Effect of treatment on OEC (ATE) from linear regression: {100*model.params['treatment[T.experiment]']:.5f}"
)
print(f"P-value from linear regression: {model.pvalues['treatment[T.experiment]']:.5f}")

print(f"\nEffect of treatment on OEC (ATE) from t-test: {ex11_oec_ate:.5f}")
print(f"P-value from t-test: {ex11_oec_pvalue:.5f}")

Effect of treatment on OEC (ATE) from linear regression: -1.58877
P-value from linear regression: 0.13194

Effect of treatment on OEC (ATE) from t-test: 1.58877
P-value from t-test: 0.13194


> As we can see, the results obtained using the t-test and linear regression are identical.

### Exercise 14

Now add indicator variables for the date of each observation. Do the standard errors on your `treatment` variable change? If so, in what direction?

Store your new standard error in `ex14_se_treatment`. Round your answer to 4 decimal places.

You should have found that your standard errors decreased by about 30\%—this is why, although just comparing means *works*, if you have additional variables adding them to your analysis can be helpful (all the usual rules for model specification apply — for example, you still want to be careful about overfitting, which one could argue is maybe part of what's happening here). 

In many other cases, the effect of adding controls is likely to be larger — the date indicators we added to our data are perfectly balanced between treatment and control, so we aren't adding a lot of data to the model by adding them as variables. They're accounting for some day-to-day variation (presumably in the types of people coming to the site), but they aren't controlling for any residual baseline differences the way a control like "gender" or "age" might (since those kind of individual-level attributes will never be perfectly balanced across treatment and control). 

In [18]:
model = smf.ols("oec_churn_ratio ~ treatment + C(Date)", data=data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        oec_churn_ratio   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     3.962
Date:                Mon, 18 Mar 2024   Prob (F-statistic):           0.000978
Time:                        12:10:25   Log-Likelihood:                 126.29
No. Observations:                  46   AIC:                            -204.6
Df Residuals:                      22   BIC:                            -160.7
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [19]:
ex14_se_treatment = model.bse["treatment[T.experiment]"]
results["ex14_se_treatment"] = round(ex14_se_treatment, 4)
print(
    f"The new standard error for the treatment variable after adding indicator variables for the date is: {ex14_se_treatment:.4f}"
)

The new standard error for the treatment variable after adding indicator variables for the date is: 0.0066


> Now, by adding indicator variables for the date of each observation, we can observe that the standard errors for my treatment variable have decreased from 0.010 to 0.007

### Exercise 15

Does this result have any impact on the recommendations you would offer Udacity?

> Yes, this result has a significant impact on the recommendations we would offer Udacity. Prior to adding date variables to the linear regression model, we didn't have sufficient evidence to confidently suggest that the treatment had a significant effect on the OEC. However, after including these additional variables, we observed a significant reduction in the p-value associated with the treatment coefficient, indicating that there is now sufficient evidence to assert that the treatment has a real impact on reducing the student dropout rate.
>
>Based on this result, our recommendation would be for Udacity to consider implementing the proposed website change, as it appears to have a positive and significant impact on improving the student experience by reducing frustration caused by time limitations during the free trial period. However, we should also consider whether this change is negatively affecting our guardrail metric, as we don't want to significantly reduce the number of students continuing past the free trial.

In [20]:
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.1158,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46,
 'ex11_oec_ate': 1.5888,
 'ex11_oec_pvalue': 0.1319,
 'ex11_guard_ate': 0.4897,
 'ex11_guard_pvalue': 0.5928,
 'ex14_se_treatment': 0.0066}

In [21]:
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}